In [27]:
import os
os.chdir("/workspaces/dev")

In [ ]:
from tensorflow.keras import layers, models, activations, losses, optimizers, metrics, regularizers
from tensorflow.keras.utils import Sequence

In [ ]:
class AlexNet():
  PADDING='same'
  WEIGHT_DECAY=0.0005
  
  def __init__(self, input_shape=(277, 277, 3), num_classes=1000):
    self._model = AlexNet.__create(input_shape, num_classes)
  
  def __create(input_shape, num_classes):
    model = models.Sequential([
      layers.Input(shape=input_shape),

      layers.Conv2D(filters=96, kernel_size=11, strides=4, padding=AlexNet.PADDING, kernel_regularizer=regularizers.l2(AlexNet.WEIGHT_DECAY)),
      layers.LayerNormalization(), 
      layers.Activation(activations.relu),
      layers.MaxPooling2D(pool_size=(3,3), strides=2),

      layers.Conv2D(filters=256, kernel_size=5, padding=AlexNet.PADDING, kernel_regularizer=regularizers.l2(AlexNet.WEIGHT_DECAY)),
      layers.LayerNormalization(),
      layers.Activation(activations.relu),
      layers.MaxPooling2D(pool_size=(3,3), strides=2),

      layers.Conv2D(filters=384, kernel_size=3, padding=AlexNet.PADDING, activation=activations.relu, kernel_regularizer=regularizers.l2(AlexNet.WEIGHT_DECAY)), 

      layers.Conv2D(filters=384, kernel_size=3, padding=AlexNet.PADDING, activation=activations.relu, kernel_regularizer=regularizers.l2(AlexNet.WEIGHT_DECAY)), 

      layers.Conv2D(filters=256, kernel_size=3, padding=AlexNet.PADDING, activation=activations.relu, kernel_regularizer=regularizers.l2(AlexNet.WEIGHT_DECAY)), 
      layers.MaxPooling2D(pool_size=(3,3), strides=2),

      layers.Flatten(),

      layers.Dense(4096, activation=activations.relu, kernel_regularizer=regularizers.l2(AlexNet.WEIGHT_DECAY)),
      layers.Dropout(0.5),

      layers.Dense(4096, activation=activations.relu, kernel_regularizer=regularizers.l2(AlexNet.WEIGHT_DECAY)),
      layers.Dropout(0.5),
      
      layers.Dense(num_classes, activation=activations.softmax)
    ])
    
    model.compile(
      loss = losses.CategoricalCrossentropy(),
      optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9),
      metrics= [metrics.Accuracy()]
    )

    return model
    
  def train(self, train_data, epochs=90, jobs=4, validation_data=None):
    return self._model.fit(
      train_data,
      epochs=epochs,
      workers = jobs,
      validation_data=validation_data,
      use_multiprocessing = True
    )

  def predict(self, x):
    return self._model.predict(x)

  # Property
  @property
  def model(self):
    return self._model
    

In [ ]:
BATCH_SIZE = 128
INPUT_SHAPE = (277, 277, 3)

In [31]:
from image_recognition.ILSVRC import ILSVRC
from data_loader import DataGenerator 

In [ ]:
label_map = ILSVRC.label_dict()
train_paths, train_labels = ILSVRC.train_paths_and_labels()
train_data = DataGenerator(train_paths, train_labels, BATCH_SIZE, INPUT_SHAPE)

In [ ]:
alexnet = AlexNet(input_shape=INPUT_SHAPE, num_classes=len(label_map))

In [34]:
isinstance(train_data, Sequence)

True

In [ ]:
alexnet.train(train_data, jobs=10)

TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'workers'